# PyTorch 基础 : 自动求导
深度学习的算法本质上是通过反向传播求导数，而PyTorch的autograd模块则实现了此功能。在Tensor上的所有操作，autograd都能为它们自动提供微分，避免了手动计算导数的复杂过程。

***从0.4起, Variable 正式合并入Tensor, Variable 本来实现的自动微分功能，Tensor就能支持。读者还是可以使用Variable(tensor), 但是这个操作其实什么都没做。***

所以，以后的代码建议直接使用Tensor，因为官方文档中已经将Variable设置成过期模块

要想使得Tensor使用autograd功能，只需要设置tensor.requries_grad=True

In [1]:
# 首先要引入相关的包
import torch
#打印一下版本
torch.__version__

'1.0.1'

在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [2]:
x = torch.rand(5, 5, requires_grad=True)
x

tensor([[0.4464, 0.1787, 0.3355, 0.6140, 0.0426],
        [0.4460, 0.7586, 0.6637, 0.7720, 0.3946],
        [0.8108, 0.0172, 0.1787, 0.9883, 0.5127],
        [0.3588, 0.4072, 0.4920, 0.5452, 0.2046],
        [0.7553, 0.6036, 0.1825, 0.4376, 0.0786]], requires_grad=True)

In [3]:
y = torch.rand(5, 5, requires_grad=True)
y

tensor([[0.5546, 0.5759, 0.3629, 0.3170, 0.6898],
        [0.8963, 0.7424, 0.8629, 0.4416, 0.3023],
        [0.1882, 0.0293, 0.6739, 0.3843, 0.5960],
        [0.0642, 0.3107, 0.0492, 0.7061, 0.3869],
        [0.6987, 0.8289, 0.5856, 0.9319, 0.1769]], requires_grad=True)

我们看到 该张量的grad_fn已经被赋予了一个新的函数。下面我们来调用反向传播函数，计算其梯度

In [4]:
z=torch.sum(x+y)
z

tensor(23.5815, grad_fn=<SumBackward0>)

## 简单的自动求导

In [5]:
z.backward()

In [11]:
#看一下x和y的梯度
print(x.grad,y.grad)

tensor([[0.2899, 0.0575, 0.3950, 0.4017, 1.3933],
        [1.8113, 0.1749, 1.3603, 1.1894, 1.2264],
        [1.1217, 0.6410, 0.5550, 0.3118, 1.5538],
        [1.4677, 1.9182, 1.0252, 1.5648, 0.7733],
        [1.3649, 0.0703, 1.3135, 0.4130, 1.0332]]) tensor([[4.5399e-02, 2.1779e+00, 4.4259e-02, 1.1141e-01, 2.6349e+00],
        [2.2494e+00, 1.7009e+00, 1.0824e+00, 3.5850e-01, 1.3431e-03],
        [8.7195e-01, 1.8007e+00, 7.6888e-01, 2.8919e+00, 3.2491e-02],
        [7.1151e-01, 2.6040e+00, 2.9655e+00, 1.2792e+00, 3.9674e-02],
        [4.3596e-01, 1.9133e+00, 2.4896e-02, 1.3848e-01, 1.7865e-01]])


## 复杂的自动求导

In [40]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
z

tensor([[0.1132, 0.7330, 0.9611, 0.5110, 0.3962],
        [0.5945, 0.3243, 0.0871, 0.4854, 0.0358],
        [0.6911, 0.7867, 0.5989, 0.4869, 1.1093],
        [0.8742, 0.0144, 1.3683, 0.7226, 0.5616],
        [0.5654, 0.0681, 0.0434, 0.2456, 0.0208]], grad_fn=<AddBackward0>)

In [44]:
# #我们的返回值不是一个scalar，所以需要输入一个大小相同的张量作为参数，
# # 这里我们用ones_like函数根据x生成一个张量
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
print(2*x)
z.backward(torch.ones_like(x))
print(2*x==x.grad)
print(x.grad)


tensor([[1.0044, 0.1193, 1.6177, 0.7746, 0.7643],
        [1.1370, 0.3680, 1.0422, 1.9593, 1.8759],
        [1.9979, 0.5522, 0.6762, 1.2147, 0.7973],
        [0.1568, 1.0888, 1.7460, 1.6865, 1.5753],
        [0.3989, 1.3811, 1.3558, 0.6002, 0.4094]], grad_fn=<MulBackward0>)
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.uint8)
tensor([[1.0044, 0.1193, 1.6177, 0.7746, 0.7643],
        [1.1370, 0.3680, 1.0422, 1.9593, 1.8759],
        [1.9979, 0.5522, 0.6762, 1.2147, 0.7973],
        [0.1568, 1.0888, 1.7460, 1.6865, 1.5753],
        [0.3989, 1.3811, 1.3558, 0.6002, 0.4094]])


In [45]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
print(3*y**2)
z.backward(torch.ones_like(y))

print(3*y**2==y.grad)
print(y.grad)

tensor([[8.2078e-02, 8.1681e-02, 2.7227e+00, 5.3312e-01, 5.9085e-01],
        [2.5563e+00, 1.3165e+00, 3.2854e-01, 1.7354e-03, 1.7892e+00],
        [2.1195e+00, 2.5040e+00, 1.4823e+00, 2.0597e+00, 6.7571e-01],
        [1.7393e-01, 2.3105e+00, 1.7602e+00, 7.9456e-01, 1.9135e-01],
        [3.6385e-03, 3.0260e-02, 2.7035e+00, 6.5360e-01, 2.1762e-01]],
       grad_fn=<MulBackward0>)
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.uint8)
tensor([[8.2078e-02, 8.1681e-02, 2.7227e+00, 5.3312e-01, 5.9085e-01],
        [2.5563e+00, 1.3165e+00, 3.2854e-01, 1.7354e-03, 1.7892e+00],
        [2.1195e+00, 2.5040e+00, 1.4823e+00, 2.0597e+00, 6.7571e-01],
        [1.7393e-01, 2.3105e+00, 1.7602e+00, 7.9456e-01, 1.9135e-01],
        [3.6385e-03, 3.0260e-02, 2.7035e+00, 6.5360e-01, 2.1762e-01]])


我们可以使用`with torch.no_grad()`上下文管理器临时禁止对已设置requires_grad=True的张量进行自动求导。这个方法在测试集计算准确率的时候会经常用到，例如

In [43]:
with torch.no_grad(): 
    print((x +y*2).requires_grad)

False
